In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
import os
os.chdir("/content/gdrive/My Drive/dlp19/proje")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
from random import sample
import pickle, cv2
#from scipy.misc import imresize
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import keras as keras
import keras_applications as keras_applications
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
import h5py


Using TensorFlow backend.


In [0]:
keras.__version__

'2.2.4'

In [0]:
seed = 1337
np.random.seed(seed)

epochs = 5
batch_size = 25
input_shape = (224, 224, 3)
 
data = np.load('/content/gdrive/My Drive/dlp19/proje/images_labels_224.npz')
x = data['x']
y = data['y']
n_classes = len(np.unique(y))    


In [0]:
def train_validation_split(x, y):
    # split data into training and test sets
    X_training, X_test, y_training, y_test = train_test_split(x, y, stratify=y, test_size= 0.1, random_state=1337)

    # split training into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_training, y_training, stratify=y_training, random_state=42)

    return X_train, X_val, X_test, y_train, y_val, y_test



In [0]:
X_train, X_val, X_test, y_train, y_val, y_test = train_validation_split(x, y)
print('Train data shape: ', X_train.shape) # 90% - 4752 (3564 + 1188)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape) # 10% - 528
print('Test labels shape: ', y_test.shape)

Train data shape:  (3564, 224, 224, 3)
Train labels shape:  (3564,)
Validation data shape:  (1188, 224, 224, 3)
Validation labels shape:  (1188,)
Test data shape:  (528, 224, 224, 3)
Test labels shape:  (528,)


In [0]:
def one_hot(y_train, y_val, y_test, n_classes):
    y_train = np_utils.to_categorical(y_train, n_classes)
    y_val = np_utils.to_categorical(y_val, n_classes)
    y_test = np_utils.to_categorical(y_test, n_classes)
    return y_train, y_val, y_test


In [0]:
y_train, y_val, y_test = one_hot(y_train, y_val, y_test, n_classes)

print(y_train)
print(y_val)
print(y_test)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [0]:
base_model2 = keras_applications.resnet.ResNet101(include_top = False, input_shape=input_shape, weights = 'imagenet', 
        backend = keras.backend, layers = keras.layers, models = keras.models, utils = keras.utils)

x2 = base_model2.output
x2 = GlobalAveragePooling2D()(x2)

predictions = Dense(n_classes, activation= 'softmax')(x2)
model2 = Model(inputs = base_model2.input, outputs = predictions)

model2.summary()

W0707 17:38:17.265438 140128931501952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0707 17:38:17.279366 140128931501952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0707 17:38:17.285400 140128931501952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0707 17:38:17.310001 140128931501952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0707 17:38:17.310976 1401289315

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________________
conv1_relu

In [0]:
# save the best model only
filepath3 ='/content/gdrive/My Drive/dlp19/proje/model_resnet101_withoutdropout.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath3, 
                                             monitor='val_acc', 
                                             verbose=1, 
                                             save_best_only=True, 
                                             mode='max')
callbacks_list = [checkpoint]


In [0]:
adam = Adam(lr=0.001)
model2.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(loss=keras.losses.categorical_crossentropy,
              #optimizer=SGD(lr=0.1, momentum=0, decay=0, nesterov=False),
              #metrics=['accuracy'])

W0707 17:38:35.375886 140128931501952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
batch_size = 65
epochs = 40
history = model2.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=callbacks_list,
                    verbose=1,
                    validation_data=(X_val, y_val))
    
    

W0707 17:38:35.559396 140128931501952 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 3564 samples, validate on 1188 samples
Epoch 1/40
3564/3564 [==============================] - 111s 31ms/step - loss: 2.1686 - acc: 0.2845 - val_loss: 3.9744 - val_acc: 0.2071

Epoch 00001: val_acc improved from -inf to 0.20707, saving model to /content/gdrive/My Drive/dlp19/proje/model_resnet101_withoutdropout.hdf5
Epoch 2/40
3564/3564 [==============================] - 82s 23ms/step - loss: 1.7243 - acc: 0.3976 - val_loss: 7.7182 - val_acc: 0.1019

Epoch 00002: val_acc did not improve from 0.20707
Epoch 3/40
3564/3564 [==============================] - 82s 23ms/step - loss: 1.5355 - acc: 0.4624 - val_loss: 2.8465 - val_acc: 0.2854

Epoch 00003: val_acc improved from 0.20707 to 0.28535, saving model to /content/gdrive/My Drive/dlp19/proje/model_resnet101_withoutdropout.hdf5
Epoch 4/40
3564/3564 [==============================] - 82s 23ms/step - loss: 1.3233 - acc: 0.5362 - val_loss: 3.2049 - val_acc: 0.1961

Epoch 00004: val_acc did not improve from 0.28535
Epoch 5/40
3564/35

In [0]:
history.history["val_acc"]


In [0]:
model2.load_weights(filepath3)
score2 = model2.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [0]:
plt.subplot(2, 1, 1)
plt.title('Training loss')
plt.plot(history.history["loss"], 'o')
plt.xlabel('Iteration')

plt.subplot(2, 1, 2)
plt.title('Accuracy')
plt.plot(history.history["acc"], '-o', label='train')
plt.plot(history.history["val_acc"], '-o', label='val')
plt.plot([0.5] * len(history.history["val_acc"]), 'k--')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.gcf().set_size_inches(15, 12)
plt.show()